#### Zadanie:

Doplňte cesty do očisteného logovacieho súboru, v ktorom boli identifikovaný používatelia a sedenia pomocou metódy STT_Mean.

Odovzdajte zdrojový kód programu + logovací súbor doplnený o identifikáciu používateľov (v jednom zip).

In [1]:
import pandas as pd
from urllib.parse import urlparse

In [2]:
file = pd.read_csv("12_Identifikácia_sedení_SLength.csv", escapechar = ";", encoding ="utf-8", index_col = False)

In [3]:
# I will use here www.example.com, but it referes to your domain name, which access logs are being examined
# (case specific)

# we need to create same structure of the data for the one we are searching 
# there are 2 ways. Either transform Request by adding https://www.example.com to each or delete
# the https://www.example.com from the referer. This way we will normalize them and can compare at latter instance

# one more rule is added., After trial and error I found that referer is one of 3 which can be treated as same:
# 1. https://www.example.com/
# 2. https://example.com
# 3. http://www.example.com

new_referer = []
for refer in file['Referer']:
    if (pd.isna(refer) or not refer.startswith("https://www.example.com/")
    or not refer.startswith("https://example.com/")
    or not refer.startswith("http://www.example.com/")):
        new_referer.append(refer)
    else:        
        referer_path = urlparse(refer).path + urlparse(refer).params + urlparse(refer).query + urlparse(refer).fragment
        new_referer.append(referer_path)


In [4]:
file['Referer'] = new_referer

In [5]:
df_RLength = file

rows = df_RLength.iloc[0:0]
RLength = 0
df_temp = df_RLength.iloc[0:0]

for index, row in df_RLength.iterrows():
    if(row.RLength == RLength):
        if(len(rows) and (rows['Request Uri'].iloc[-1] == row['Referer'])):
            rows = rows.append(row,ignore_index=True)
        elif((row['Request Uri'] in list(rows['Request Uri'])) and (not pd.isna(row['Referer'])) and str(row['Referer'].startswith('/'))):
            last_index = rows['Request Uri'].where(rows['Request Uri']==row['Request Uri']).last_valid_index()
            if (rows['Referer'].iloc[last_index] == row['Referer']):
                part_refer = list(rows['Request Uri'].iloc[last_index:])
                part_refer.reverse()
                part_url = list(rows['Referer'].iloc[last_index:])
                part_url.reverse()
                part_tuples = list(zip(part_url,part_refer))
                for item in part_tuples:
                    new_row = rows.iloc[-1]
                    new_row['Request Uri'] = item[0]
                    new_row['Referer'] = item[1]
                    new_row['UNIXTIME'] += 1
                    new_row.length = 1
                    df_temp = df_temp.append(new_row,ignore_index=True)
                rows = df_RLength.iloc[0:0]
            else:
                rows = df_RLength.iloc[0:0]
                try:
                    if row['Referer'].startswith('/'):
                        rows = rows.append(row,ignore_index=True)
                except:
                    continue
        else:
            rows = df_RLength.iloc[0:0]
            try:
                if row['Referer'].startswith('/'):
                    rows = rows.append(row,ignore_index=True)
            except:
                continue
    else:      
        RLength = row.RLength
        rows = df_RLength.iloc[0:0]
        try:
            if row['Referer'].startswith('/'):
                rows = rows.append(row,ignore_index=True)
        except:
            continue

<ipython-input-5-d2beeae2472d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['Request Uri'] = item[0]
<ipython-input-5-d2beeae2472d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['Referer'] = item[1]
<ipython-input-5-d2beeae2472d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row['UNIXTIME'] += 1


In [6]:
#df_temp.cookie = 'new log'
df_RLength = df_RLength.append(df_temp,ignore_index=True)
df_RLength.sort_values(by=['Remote Host', 'User-Agent', 'UNIXTIME'],ascending=[True, True, True], inplace=True)
df_RLength

,USER_ID,SEQUENCE_ID,Remote Host,Remote Logname,Remote User,Request Time,UNIXTIME,Length,RLength,STT_mean,STT_Q,SLength,Request Method,Request Uri,Request Protocol,Final Status Code,Bytes Sent,Referer,User-Agent
0,1,1,10.160.0.12,NaN,NaN,2020-02-19 08:15:24+01:00,1.582097e+09,NaN,1,1,1,1,GET,/,HTTP/1.1,200,28549,NaN,Mozilla/5.0 (Windows NT 5.1 rv:15.0) Gecko/201...
1,2,2,10.160.0.126,NaN,NaN,2020-02-19 14:45:24+01:00,1.582120e+09,NaN,2,2,2,2,GET,/,HTTP/1.1,200,28581,NaN,Mozilla/5.0 (Windows NT 5.1 rv:15.0) Gecko/201...
2,3,3,10.160.0.127,NaN,NaN,2020-02-19 13:02:33+01:00,1.582114e+09,NaN,3,3,3,3,GET,/,HTTP/1.1,200,28565,NaN,Mozilla/5.0 (Windows NT 5.1 rv:15.0) Gecko/201...
3,4,4,10.160.0.13,NaN,NaN,2020-02-19 10:49:25+01:00,1.582106e+09,NaN,4,4,4,4,GET,/,HTTP/1.1,200,28549,NaN,Mozilla/5.0 (Windows NT 5.1 rv:15.0) Gecko/201...
4,5,5,10.160.0.166,NaN,NaN,2020-02-19 09:32:19+01:00,1.582101e+09,1670.0,5,5,5,5,GET,/,HTTP/1.1,200,28507,NaN,Mozilla/5.0 (Windows NT 10.0 Win64 x64) AppleW...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20051,3911,4896,95.47.178.141,NaN,NaN,2020-02-19 21:28:06+01:00,1.582144e+09,NaN,7132,6665,7406,6038,GET,/,HTTP/1.1,200,28426,NaN,Mozilla/5.0 (Windows NT 10.0 Win64 x64) AppleW...
20052,3912,4897,95.62.27.140,NaN,NaN,2020-02-20 02:30:45+01:00,1.582162e+09,NaN,7133,6666,7407,6039,GET,/,HTTP/1.1,200,28330,NaN,Mozilla/5.0 (iPhone CPU iPhone OS 12_3_1 like ...
20053,3913,4898,95.82.135.62,NaN,NaN,2020-02-19 18:12:25+01:00,1.582132e+09,0.0,7134,6667,7408,6040,GET,/informacie/vyberove-konania,HTTP/1.1,301,4336,https://www.minedu.sk/volne-pracovne-miesta-na...,Mozilla/5.0 (Windows NT 10.0 Win64 x64) AppleW...
20054,3913,4898,95.82.135.62,NaN,NaN,2020-02-19 18:12:25+01:00,1.582132e+09,NaN,7134,6667,7408,6040,GET,/verejnost/uradna-vyveska/vyberove-konania,HTTP/1.1,200,29543,https://www.minedu.sk/volne-pracovne-miesta-na...,Mozilla/5.0 (Windows NT 10.0 Win64 x64) AppleW...


In [7]:
# # writing the file to new csv, escape char ";" to not get User-Agent divided on export/write
# file.to_csv("15_Dopĺňanie_ciest_RLength.csv", encoding="utf-8",  escapechar=";", index=False)